# Find dataset

In [1]:
# in this notebook we'll only get one of the files (the Oscar one) for the sake of simplicity and performance
# !wget -c https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt

In [2]:
# import pandas as pd
# df = pd.read_csv("../dnabert/examples/unprocessed_data/unlabeled_sgrna_fixed.csv")

# with open("../dnabert/examples/unprocessed_data/unlabeled_sgrna_fixed.txt","w") as f:
#     for i in df["sgRNA"]:
#         f.write(i+"\n")
        


In [3]:
# !cp ../dnabert/examples/unprocessed_data/unlabeled_sgrna_fixed.txt .

In [4]:
# from pathlib import Path

# from tokenizers import ByteLevelBPETokenizer

# paths = [str(x) for x in Path(".").glob("**/*.txt")]


# print(paths)


# # Initialize a tokenizer
# tokenizer = ByteLevelBPETokenizer()

# # Customize training
# tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
#     "<s>",
#     "<pad>",
#     "</s>",
#     "<unk>",
#     "<mask>",
# ])

In [5]:
# tokenizer.save_model("/mnt/d/M3/Projects/BCB/crisprBert/Models/crisprBert")

In [6]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "/mnt/d/M3/Projects/BCB/crisprBert/Models/crisprBert/vocab.json",
    "/mnt/d/M3/Projects/BCB/crisprBert/Models/crisprBert/merges.txt",
)

In [7]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [8]:
tokenizer.encode("CGCCGCCGCTTTCGGTGATGAGG")

Encoding(num_tokens=6, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [9]:
tokenizer.encode("CGCCGCCGCTTTCGGTGATGAGG").tokens

['<s>', 'CGCCGCC', 'GCTTT', 'CGGT', 'GATGAGG', '</s>']

In [10]:
import torch
torch.cuda.is_available()

True

In [11]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)
config.to_json_file("/mnt/d/M3/Projects/BCB/crisprBert/Models/crisprBert/config.json")

In [12]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("/mnt/d/M3/Projects/BCB/crisprBert/Models/crisprBert", max_len=512)

In [13]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [14]:
model.num_parameters()


83504416

In [15]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./unlabeled_sgrna_fixed.txt",
    block_size=128,
)

/home/woreom/anaconda3/envs/crisprBert/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:121: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [16]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [19]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="Models/crisprBert/",
    overwrite_output_dir=True,
    num_train_epochs=50,
    per_gpu_train_batch_size=8,
    save_steps=10000,
    save_total_limit=10,
    prediction_loss_only=False, 
    do_train = True, 
    do_eval = True,
    do_predict = False,
    per_gpu_eval_batch_size = 6,
    learning_rate = 4e-4,
    weight_decay = 0.01, 
    adam_beta1 = 0.9,
    adam_beta2 = 0.98,
    adam_epsilon = 1e-6 ,
    max_grad_norm =1.0,
    logging_dir = "Models/crisprBert/log.txt",
    logging_first_step = True,
    logging_steps =10000,
    evaluation_strategy = "steps",
    logging_nan_inf_filter = False, 
    fp16 = False, 
    eval_steps = 10000,  
    disable_tqdm = False, 
    load_best_model_at_end = True, 
    metric_for_best_model = "eval_loss" )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 1023845
  Num Epochs = 50
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6399050


Step,Training Loss,Validation Loss


In [ ]:
a = 10_000
type(a)